In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from toolkit.nlp import NLPTrainingConfig
path = '../env_run/outputs/optimal_checkpoint'
config = NLPTrainingConfig.load(path)

model = AutoModelForCausalLM.from_pretrained(path, trust_remote_code=True, torch_dtype='auto')
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)

/usr/local/python3.11.2/lib/python3.11/site-packages/torchmetrics/utilities/imports.py:24: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _PYTHON_LOWER_3_8 = LooseVersion(_PYTHON_VERSION) < LooseVersion("3.8")


2023-10-17 18:39:20,420 <INFO> Configuration: 👻 Custom attributes:
2023-10-17 18:39:20,422 <INFO> Configuration:    cache_dir=None
2023-10-17 18:39:20,423 <INFO> Configuration:    model_revision=main
2023-10-17 18:39:20,423 <INFO> Configuration:    use_fast_tokenizer=True
2023-10-17 18:39:20,424 <INFO> Configuration:    use_auth_token=False
2023-10-17 18:39:20,425 <INFO> Configuration:    preprocessing_num_workers=8
2023-10-17 18:39:20,425 <INFO> Configuration:    max_seq_length=2048
2023-10-17 18:39:20,426 <INFO> Configuration:    deepspeed_config=./ds_zero3_offload.hjson
2023-10-17 18:39:20,427 <INFO> Configuration:    total_steps_num=470
2023-10-17 18:39:20,427 <INFO> Configuration:    steps_per_epoch=94


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
import pandas as pd
df = pd.read_json(config.val_file_path, lines=True)
df.head(2)

,instruction,input,output,query
0,,徐怀钰参加综艺节目乘风后，引起了广泛关注。她曾经经历过家庭贫困和爷爷的心理问题，复出时效果并...,\n徐怀钰近期参加综艺节目《乘风》复出，享受舞台找回自我。\n徐怀钰的经历充满波折，家庭贫困...,None
1,,南京同曦官宣引进法尔和布莱克尼 法尔和布莱克尼加入球队。随着南京同曦更换赞助商之后，球队在引...,同曦男篮宣布引进法尔和布莱克尼，给球队的阵容注入了新的活力。他们的加入有望帮助球队突破历史，...,媒体人谈同曦新双外：针对性很强


In [4]:
# import pandas as pd
# df = pd.read_json("data/hot_finetune_data/train/all.json", lines=True)
# df.head(2)

In [19]:
from build_dataset import prompt_transfer
import torch
torch.cuda.set_device(0)

model.cuda()
for idx, row in df.iterrows():
    inputs = prompt_transfer(row['input'], row['output'])
    print(inputs)
    batch = tokenizer(inputs, return_tensors='pt')
    batch = {key: value.cuda() for key, value in batch.items()}
    outputs = model.generate(**batch, **config.generate_kwargs)
    texts = []
    for idx, output in enumerate(outputs):
        texts.append(tokenizer.decode(output[batch['input_ids'][idx].size(0):], skip_special_tokens=True))
    # texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    break

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
给定文本：
徐怀钰参加综艺节目乘风后，引起了广泛关注。她曾经经历过家庭贫困和爷爷的心理问题，复出时效果并不理想。然而，她在乘风中逐渐找回了自我，与秋瓷炫等人成为了好友，并和黄丽玲合作。虽然她不是王心凌，但乘风提供了一个机会，让她重新找回自我，比输赢更重要。徐怀钰的经历令人心疼。她从小家庭贫困，爷爷对她不好，留下了永远的心理创伤。这些年，她虽然一直在努力复出，但效果都不好。在参加乘风时，一公因路灯下的小姑娘被抨击而难过。好在Ella等人一起安慰她，她才挺过来。五公时，慢热的徐怀钰终于找到了感觉。秋瓷炫都为徐怀钰的变化感觉高兴，看得出她开始享受舞台了。这次居然主动和黄丽玲合作，要知道，这两位实力唱将合作，应该会很有很多火花。黄丽玲也很期待。徐怀钰不是王心凌，但她的复出并不仅仅是为了赢。对于粉丝来说，乘风只是一个综艺，能让徐怀钰找回自我，比什么输赢都重要。乘风让徐怀钰逐渐敞开心扉，与秋瓷炫等人成为了好友，也让她重新找到了表演的乐趣。她还和黄丽玲合作过，黄丽玲也很期待。有些人可能对徐怀钰的表现有所期待，毕竟她曾经是一位著名歌手，但输赢对她来说似乎已经不再重要了。
生成300字左右视频文案。

### Response: 


In [22]:
print(texts[0])


徐怀钰参加综艺节目《乘风》引发关注，她曾是家庭贫困和爷爷心理问题的受害者，复出效果不佳。
徐怀钰从小家庭贫困，爷爷对她不好，留下了永远的心理创伤。虽然她一直努力复出，但效果都不好。参加《乘风》时，一公因路灯下的小姑娘被抨击而难过，好在Ella等人安慰她，她才挺过来。五公时，慢热的徐怀钰终于找到了感觉，秋瓷炫都为她变化感到高兴，她开始享受舞台了。甚至主动与黄丽玲合作，要知道，这两位实力唱将合作，应该会很有很多火花。黄丽玲也很期待。
徐怀钰虽然不是王心凌，但她的复出并不仅仅是为了赢。对于粉丝来说，《乘风》只是一个综艺，能让徐怀钰找回自我，比什么输赢都重要。


In [21]:
print(row['output'])


徐怀钰近期参加综艺节目《乘风》复出，享受舞台找回自我。
徐怀钰的经历充满波折，家庭贫困、爷爷的心理问题，这些年一直在努力付出。虽然在《乘风》的舞台上复出效果不理想，但我们也看到了徐怀钰逐渐找回自我，这比比赛的输赢更重要。在《乘风》中，她与秋瓷炫、黄丽玲等人成为了好友，并携手合作。虽然她没有王心凌的机遇，但《乘风》也为她提供了一个重要的机会。她因一公被抨击而感到难过，好在有Ella等人的安慰与支持，让她挺过了难关。五公时，慢热的徐怀钰终于找到了感觉，秋瓷炫也为她的变化感到高兴，看出她开始享受舞台了。这次，徐怀钰主动和黄丽玲合作，无疑将带来许多火花。
作为徐怀钰复出的舞台，《乘风》让徐怀钰找回终于找回了自我。让我们一起期待徐怀钰更多精彩表现！


In [43]:
from toolkit.training import Evaluator
from load_data_fn import load_data_fn
from toolkit.enums import Split
from toolkit.nlp import TextDataset
from toolkit.metric import rouge, bleu
def eval_callback(all_labels, all_logits, mean_loss):
    metric = rouge(all_logits, all_labels, "zh", ("rouge1", "rouge2", "rougeL"))
    metric.update(bleu(all_logits, all_label, 'zh', ("bleu1", "bleu2", "bleu3", "bleu4")))
    return metric, all_labels, all_logits

config.infer_batch_size=8
dataset = TextDataset.from_file(config.val_file_path, tokenizer, Split.VALIDATION, config, load_data_fn, use_cache=False, debug=True)
evaluator = Evaluator('generate', config, model, dataset, eval_callback, tokenizer=tokenizer, extral_args_evaluation=None)
metric, all_label, all_logits = evaluator.eval(Split.VALIDATION, 1)

2023-10-17 20:00:26,638 <DEBUG> TextDataset: ⏳ Loading VALIDATION dataset ...
2023-10-17 20:00:26,639 <INFO> TextDataset: Model max length: 4096


Tokenize input texts:   0%|          | 0/8 [00:00<?, ?it/s]

2023-10-17 20:00:26,717 <DEBUG> TextDataset: ⌛ Loading VALIDATION data takes 0.08 sec.
2023-10-17 20:00:26,718 <INFO> TextDataset: Total data: 8
2023-10-17 20:00:26,719 <INFO> TextDataset: Max length of input: 974
2023-10-17 20:00:26,719 <INFO> TextDataset: Max length of label: -1


VALIDATION:   0%|          | 0/4 [00:00<?, ?batch/s]

Calculating rouge:   0%|          | 0/8 [00:00<?, ?it/s]

In [44]:
print(metric)
for i in range(8):
    print("-----label------")
    print(all_label[i])
    print("-----pred------")
    print(all_logits[i])
    print('########################################')

{'rouge1': 0.5903821736574173, 'rouge2': 0.3561397325247526, 'rougeL': 0.36820829287171364, 'bleu1': 0.5656016315431679, 'bleu2': 0.43774224834912157, 'bleu3': 0.35235439239397365, 'bleu4': 0.29335614007602784}
-----label------

徐怀钰近期参加综艺节目《乘风》复出，享受舞台找回自我。
徐怀钰的经历充满波折，家庭贫困、爷爷的心理问题，这些年一直在努力付出。虽然在《乘风》的舞台上复出效果不理想，但我们也看到了徐怀钰逐渐找回自我，这比比赛的输赢更重要。在《乘风》中，她与秋瓷炫、黄丽玲等人成为了好友，并携手合作。虽然她没有王心凌的机遇，但《乘风》也为她提供了一个重要的机会。她因一公被抨击而感到难过，好在有Ella等人的安慰与支持，让她挺过了难关。五公时，慢热的徐怀钰终于找到了感觉，秋瓷炫也为她的变化感到高兴，看出她开始享受舞台了。这次，徐怀钰主动和黄丽玲合作，无疑将带来许多火花。
作为徐怀钰复出的舞台，《乘风》让徐怀钰找回终于找回了自我。让我们一起期待徐怀钰更多精彩表现！
-----pred------

徐怀钰参加综艺节目《乘风》引发关注，她曾是家庭贫困和爷爷心理问题的受害者，复出效果不佳。
徐怀钰从小家庭贫困，爷爷对她不好，留下了永远的心理创伤。虽然她一直努力复出，但效果都不好。参加《乘风》时，一公因路灯下的小姑娘被抨击而难过，好在Ella等人安慰她，她才挺过来。五公时，慢热的徐怀钰终于找到了感觉，秋瓷炫都为她变化感到高兴，她开始享受舞台了。甚至主动与黄丽玲合作，要知道，这两位实力唱将合作，应该会很有很多火花。黄丽玲也很期待。
徐怀钰虽然不是王心凌，但她的复出并不仅仅是为了赢。对于粉丝来说，《乘风》只是一个综艺，能让徐怀钰找回自我，比什么输赢都重要。
########################################
-----label------
同曦男篮宣布引进法尔和布莱克尼，给球队的阵容注入了新的活力。他们的加入有望帮助球队突破历史，冲击季后赛的门票。
同曦男篮一直是CBA中唯二未进入过季后赛的球队，但这次他们给人们带来了惊

# test extral evaluator

In [69]:
config.generate_kwargs

{'do_sample': True,
 'num_beams': 1,
 'early_stopping': False,
 'use_cache': True,
 'temperature': 0.01,
 'top_k': 5,
 'top_p': 0.85,
 'diversity_penalty': 0.0,
 'repetition_penalty': 1.1,
 'length_penalty': 1.0,
 'max_new_tokens': 1024}

In [71]:
import json

with open("generate_config.json", "r", encoding='utf8') as f:
    generate_config = json.load(f)


with open("generate_config.json", "w", encoding='utf8') as f:
    json.dump(generate_config, f, indent=2)

generate_config


{'do_sample': True,
 'num_beams': 1,
 'early_stopping': False,
 'use_cache': True,
 'temperature': 1,
 'top_k': 20,
 'top_p': 1.0,
 'diversity_penalty': 0.0,
 'repetition_penalty': 1.1,
 'length_penalty': 1.0,
 'max_new_tokens': 1024}

In [72]:
from extral_evaluator import Extral_Evaluator, extra_calculate_metric_callback
from load_data_fn import load_data_fn
from toolkit.enums import Split
from toolkit.nlp import TextDataset
from toolkit.metric import self_bleu
def extra_calculate_metric_callback(all_labels, all_logits):
    return self_bleu(all_logits, 'zh', ("bleu1", "bleu2", "bleu3", "bleu4"), smoothing_level=1), all_label, all_logits

config.generate_config_file="generate_config.json"
config.re_gen_num=2
config.infer_batch_size=8


dataset = TextDataset.from_file(config.val_file_path, tokenizer, Split.VALIDATION, config, load_data_fn, use_cache=False, debug=True)
evaluator = Extral_Evaluator('generate', config, model, dataset, extra_calculate_metric_callback, tokenizer=tokenizer)
metric, all_label, all_logits  = evaluator.eval(Split.VALIDATION, 1)

2023-10-17 21:17:58,558 <DEBUG> TextDataset: ⏳ Loading VALIDATION dataset ...
2023-10-17 21:17:58,559 <INFO> TextDataset: Model max length: 4096


Tokenize input texts:   0%|          | 0/4 [00:00<?, ?it/s]

2023-10-17 21:17:58,612 <DEBUG> TextDataset: ⌛ Loading VALIDATION data takes 0.05 sec.
2023-10-17 21:17:58,613 <INFO> TextDataset: Total data: 4
2023-10-17 21:17:58,614 <INFO> TextDataset: Max length of input: 795
2023-10-17 21:17:58,615 <INFO> TextDataset: Max length of label: -1


VALIDATION:   0%|          | 0/2 [00:00<?, ?batch/s]

VALIDATION:   0%|          | 0/2 [00:00<?, ?batch/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [73]:
from pprint import pp
print(metric)
for i in range(4):
    print("-----label------")
    pp(all_label[i])
    print("-----pred------")
    pp(all_logits[i])
    print('########################################')

{'self-bleu1': 0.42270517794194007, 'self-bleu2': 0.5632167196853513, 'self-bleu3': 0.6499507325706294, 'self-bleu4': 0.708613560314383}
-----label------
('\n'
 '徐怀钰近期参加综艺节目《乘风》复出，享受舞台找回自我。\n'
 '徐怀钰的经历充满波折，家庭贫困、爷爷的心理问题，这些年一直在努力付出。虽然在《乘风》的舞台上复出效果不理想，但我们也看到了徐怀钰逐渐找回自我，这比比赛的输赢更重要。在《乘风》中，她与秋瓷炫、黄丽玲等人成为了好友，并携手合作。虽然她没有王心凌的机遇，但《乘风》也为她提供了一个重要的机会。她因一公被抨击而感到难过，好在有Ella等人的安慰与支持，让她挺过了难关。五公时，慢热的徐怀钰终于找到了感觉，秋瓷炫也为她的变化感到高兴，看出她开始享受舞台了。这次，徐怀钰主动和黄丽玲合作，无疑将带来许多火花。\n'
 '作为徐怀钰复出的舞台，《乘风》让徐怀钰找回终于找回了自我。让我们一起期待徐怀钰更多精彩表现！')
-----pred------
['\n'
 '徐怀钰参加综艺节目《乘风》引发关注，网友期待她能再次展现自己的才华。\n'
 '徐怀钰曾是家庭贫困的孩子，经历了爷爷的心理问题，复出效果不佳。然而，在《乘风》中，她逐渐找回了自我，与秋瓷炫等人成为好友，并与黄丽玲合作。虽然她不是王心凌，但《乘风》提供了一个机会，让她重新找回自我，比输赢更重要。徐怀钰的经历令人心疼，她从小家庭贫困，爷爷对她不好，留下了永远的心理创伤。尽管她一直努力复出，但效果始终不好。在参加《乘风》时，一公因路灯下的小姑娘被抨击而难过，幸好Ella等人及时安慰她，她才能挺过去。五公时，慢热的徐怀钰终于找到了感觉， '
 'Autumn也为她的变化感到高兴，看着她开始享受舞台的乐趣。这次她竟然主动与黄丽玲合作，要知道，这两位实力唱将合作，必将产生许多火花。黄丽玲也十分期待。\n'
 '尽管徐怀钰不是王心凌，但她的复出对于粉丝来说，只是一个综艺节目，能够让徐怀钰找回自我，比什么输赢都重要。',
 '\n'
 '徐怀钰参加综艺节目《乘风》引发网友热议，原因是她不再争输赢。\n'
 '徐怀钰从小家庭贫困，爷爷对她不好，给她留下了心理创